# 짧은 조언 생성용 LLM 파인튜닝

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# 기본 파일 설정 및 Cuda 확인
BASE_DIR = ""
TRAIN_FILE = os.path.join(BASE_DIR, "empathy_train.jsonl")
VALID_FILE = os.path.join(BASE_DIR, "empathy_valid.jsonl")

MODEL_NAME = "Bllossom/llama-3.2-Korean-Bllossom-3B"
EXPERIMENT_NAME = "Bllossom/llama-3.2-Korean-Bllossom-3B"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Device: {DEVICE}")

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype=torch.float16,
    device_map="auto",
    attn_implementation="flash_attention_2",
)

model = prepare_model_for_kbit_training(model)

# Mlflow 세팅

In [ ]:
import mlflow

mlflow.set_tracking_uri("file:./blossom_mlruns")
mlflow.set_experiment("blossom_Finetuning")
mlflow.end_run()

In [ ]:
def preprocess_dataset(file_path):
    dataset = load_dataset("json", data_files={"data": file_path})["data"]

    def concat_text(ex):
        text = f"{ex['input']}\n{ex['output']}{tokenizer.eos_token}"
        return {"text": text}

    dataset = dataset.map(concat_text, remove_columns=dataset.column_names)

    def tokenize_function(examples):
        return tokenizer(
            examples["text"],
            padding="max_length",
            truncation=True,
            max_length=512
        )

    tokenized = dataset.map(tokenize_function, batched=True)
    tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"])
    return tokenized

train_dataset = preprocess_dataset(TRAIN_FILE)
valid_dataset = preprocess_dataset(VALID_FILE)

# 파인튜닝 및 경량화 설정

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir=os.path.join(BASE_DIR, "blossom_finetuned"),
    
    eval_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,
    num_train_epochs=3,
    
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    
    warmup_ratio=0.05,
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    
    lr_scheduler_type="cosine",
    report_to=["mlflow"],
    run_name="blossom_empathy_lora",
    load_best_model_at_end=True,
    logging_dir="./logs",
    optim="adamw_torch"
)

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.log_params({
    "model": MODEL_NAME,
    "train_file": TRAIN_FILE,
    "valid_file": VALID_FILE,
    "batch_size": training_args.per_device_train_batch_size,
    "learning_rate": training_args.learning_rate,
    "epochs": training_args.num_train_epochs,
    "accum_steps": training_args.gradient_accumulation_steps,
    "lora_r": lora_config.r,
    "lora_alpha": lora_config.lora_alpha,
})

# ==========================
# ✅ Trainer 정의
# ==========================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ==========================
# ✅ 학습 시작
# ==========================
print("🚀 LoRA 학습 시작")
trainer.train()
trainer.save_model(os.path.join(BASE_DIR, "Advice_finetuned"))

# ==========================
# ✅ 최종 평가 및 로그
# ==========================
eval_results = trainer.evaluate()
mlflow.log_metrics(eval_results)
mlflow.end_run()

print("✅ 학습 완료 및 MLflow 로그 저장 완료")

# LoRA 가중치 병합

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

BASE_MODEL = MODEL_NAME   # ex: "Bllossom/llama-3.2-Korean-Bllossom-3B"
LORA_MODEL = os.path.join(BASE_DIR, "Advice_finetuned")  # trainer.save_model 위치
MERGED_OUTPUT = os.path.join(BASE_DIR, "Advice_finetuned_merged")

print("🔄 Base 모델 로딩...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype="auto",
)

print("🔄 LoRA weight 로딩...")
model = PeftModel.from_pretrained(
    base_model,
    LORA_MODEL,
)

print("🔁 LoRA → Base 모델로 merge 중...")
merged_model = model.merge_and_unload()   # **핵심 한 줄**

print("💾 병합된 모델 저장...")
merged_model.save_pretrained(MERGED_OUTPUT, safe_serialization=True)

print("💾 토크나이저 저장...")
tokenizer.save_pretrained(MERGED_OUTPUT)

print("🎉 LoRA 병합 완료! 저장 위치:", MERGED_OUTPUT)
